In [1]:
import torch
import argparse
import sys
import os

import torchvision
import torchvision.transforms as transforms


In [2]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])
trainset = torchvision.datasets.CIFAR10(
    root='../../../../datasets/data/cifar10/',
    train=True,
    download=True,
    transform=transform_train)

train_loader = torch.utils.data.DataLoader(
        trainset,
        batch_size=128)

transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010))
    ])

testset = torchvision.datasets.CIFAR10(
        root='../../../../datasets/data/cifar10/',
        train=False,
        download=True,
        transform=transform_test)

testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=int(len(testset) / 10),
                                         drop_last=False,
                                         shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class AverageMeter(object):
    """Record train infomation"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0.0
        self.avg = 0.0
        self.sum = 0.0
        self.count = 0.0

    def update(self, val, n=1):
        self.val = val
        self.sum += val
        self.count += n
        self.avg = self.sum / self.count
        
def evaluate(model, criterion, test_loader):
    model.eval()
    gpu = next(model.parameters()).device

    loss_ = AverageMeter()
    acc_ = AverageMeter()

    with torch.no_grad():
        for inputs, labels in test_loader:

            inputs = inputs.to(gpu)
            labels = labels.to(gpu)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs, 1)
            loss_.update(loss.item())
            acc_.update(torch.sum(predicted.eq(labels)).item(), len(labels))

    return loss_.sum, acc_.avg

In [4]:
model = torchvision.models.resnet18().cuda()

In [5]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


for epoch in range(20):
    for data,label in train_loader:

        data = data.cuda()
        label = label.cuda()

        output = model(data)

        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_, acc_ = evaluate(model, criterion, testloader)
    print(epoch, loss_, acc_)

0 15.475115180015564 0.4292
1 16.243518829345703 0.4354
2 14.03718876838684 0.4895
3 13.08228349685669 0.5283
4 12.624475359916687 0.5395
5 12.213253498077393 0.5614
6 12.198041439056396 0.567
7 11.79844081401825 0.5839


KeyboardInterrupt: 